## PTAC Data Inventory

Investigating the PTAC data inventory.

In [2]:
from redaptipy.db_clients.rdp_db_client import RdpDBClient

import pandas as pd

pd.set_option('display.max_rows', None)

In [3]:
rdp_client = RdpDBClient()

In [8]:
ptac_query = """
    SELECT 
    circuit.id,
    circuit.breaker_number,
    circuit.description, 
    equipment.name as equipment_name,
    meter.name as metermac,
    panel.name as panel_name,
    site.address1,
    site.city,
    site.lat,
    site.lng,
    site.state FROM circuit
    JOIN building_system ON circuit.building_system_id = building_system.id
    LEFT join equipment ON circuit.equipment_id = equipment.id
    JOIN meter ON circuit.meter_id = meter.id
    LEFT JOIN panel ON circuit.panel_id = panel.id
    JOIN site ON meter.site_id = site.id
    JOIN customer ON site.customer_id = customer.id
    WHERE 1=1
    AND customer.name = 'ALG Senior'
    AND building_system.name in ('HVAC', 'Other Submetered')
    AND (circuit.description ILIKE 'room%' 
        OR circuit.description ILIKE '%rm%' 
        OR circuit.description ILIKE '%wall%'
        OR circuit.description ILIKE '%tw%'
        OR circuit.description ILIKE '%ptac%'
        OR circuit.description ILIKE '%through wall%'
        OR circuit.description ILIKE '%thru wall%'
        OR equipment.name ILIKE '%hchv%')
    AND (circuit.description NOT ILIKE '%living%' OR
         circuit.description NOT ILIKE '%dining%')
    """

df = rdp_client.query(ptac_query)

df.to_excel('ALG_ptac_inventory.xlsx')
